In [ ]:
import numpy as np
from scipy.optimize import minimize

# Triangle vertices
P1 = np.array([0.0, 0.0])
P2 = np.array([1.0, 0.0])
P3 = np.array([0.0, 1.0])

def compute_xy_bounds_from_triangle(P1, P2, P3, margin=0.1):
    """
    根据三角形三个顶点，返回 (x, y) 的轴对齐 bounding box 上下界，用于优化器的 bounds。

    参数：
        P1, P2, P3 : array-like，形如 [x, y] 的三角形顶点
        margin : float，扩展边界的安全余量（默认 0.1）

    返回：
        xy_bounds : [(x_min, x_max), (y_min, y_max)]
    """
    points = np.array([P1, P2, P3])
    x_min, x_max = np.min(points[:, 0]), np.max(points[:, 0])
    y_min, y_max = np.min(points[:, 1]), np.max(points[:, 1])
    
    return [(x_min - margin, x_max + margin), (y_min - margin, y_max + margin)]

# Original vector field
# u_dec = np.array([1.5, 0.7, 0.1])
# v_orig = np.array([1.99,  -0.35, -5.0])
# epsilon = 0.3

# u_dec = [ 1.5, -0.6,  0.2]
# v_orig = [ 1.3,  0.0, -1.0]
# epsilon = 0.3

# u_orig = [ 3.87, -9.42,  5.55]
# v_orig = [ 5.23, 7.76,  2.53]
# epsilon = 1.5

# u_orig = [1.65, 0.87, 0.79] #inside
# v_orig = [-0.57, -1.76, 0.21]
# epsilon = 1.5

u = [1.55 ,0.77 ,0.69]
v = [-0.47, -1.66 , 0.31]
epsilon = 0.3

def barycentric_coords(x, y):
    return np.array([1 - x - y, x, y])

def interpolate(vec, lambdas):
    return np.dot(lambdas, vec)

def get_original_critical_point(u, v):
    A = np.array([[2, 1], [1, 2]])
    b = np.array([u[0] - u[1], v[0] - v[1]])
    x, y = np.linalg.solve(A, b)
    lambdas = barycentric_coords(x, y)
    return x, y, lambdas

# Constraint functions
def u_zero(z):
    u = z[0:3]
    x, y = z[6], z[7]
    lambdas = barycentric_coords(x, y)
    return interpolate(u, lambdas)

def v_zero(z):
    v = z[3:6]
    x, y = z[6], z[7]
    lambdas = barycentric_coords(x, y)
    return interpolate(v, lambdas)

def lambda_outside_constraint():
    return {'type': 'ineq', 'fun': lambda z: -np.min(barycentric_coords(z[6], z[7])) - 1e-5}

# Objective function
def objective(z):
    u = z[0:3]
    v = z[3:6]
    return np.sum(np.abs(u - u_orig)) + np.sum(np.abs(v - v_orig))


x0_cp, y0_cp, lambda_orig = get_original_critical_point(u_orig, v_orig)
x0 = np.concatenate([u_orig, v_orig, [x0_cp, y0_cp]])

# Variable bounds
# bounds = [(u_orig[i] - epsilon, u_orig[i] + epsilon) for i in range(3)] + \
#          [(v_orig[i] - epsilon, v_orig[i] + epsilon) for i in range(3)] + \
#          [(-2.0, 2.0), (-2.0, 2.0)]  # x, y

bounds = [(u_orig[i] - epsilon, u_orig[i] + epsilon) for i in range(3)] + \
         [(v_orig[i] - epsilon, v_orig[i] + epsilon) for i in range(3)] + \
         compute_xy_bounds_from_triangle(P1, P2, P3, margin=0.1) # in the triangle

# Constraints (u = 0, v = 0, and force outside)
constraints = [
    {'type': 'eq', 'fun': u_zero},
    {'type': 'eq', 'fun': v_zero},
    lambda_outside_constraint()
]

# Solve optimization
result = minimize(
    fun=objective,
    x0=x0,
    bounds=bounds,
    constraints=constraints,
    method='trust-constr',
    options={'disp': True, 'maxiter': 2000}
)

# Result summary
u_opt = result.x[0:3]
v_opt = result.x[3:6]
x_crit, y_crit = result.x[6], result.x[7]
lambda_vec = barycentric_coords(x_crit, y_crit)


def cp_inside(lambda_vec):
    # Check if the critical point is inside the triangle
    if np.all(lambda_vec >= 0) and np.all(lambda_vec <= 1):
        return True
    else:
        return False


def verify(u,v,new_u,new_v,epsilon):
    # check if each component of u and v is within epsilon of the original
    for i in range(3):
        if not (abs(u[i] - new_u[i]) <= epsilon and abs(v[i] - new_v[i]) <= epsilon):
            print("u or v is not within epsilon of the original.")
            return False
    # check : if original critical point is outside the triangle, new must be also outside. if original cp inside, new must be also inside
    if (cp_inside(lambda_orig) != cp_inside(lambda_vec)):
        print("Critical point inside/outside mismatch.")
        return False
    else:
        return True

print("\n========== Summary ==========")
print("Original Critical Point:     (%.4f, %.4f)" % (x0_cp, y0_cp))
print("Original Lambdas:            λ1=%.4f, λ2=%.4f, λ3=%.4f" % tuple(lambda_orig))
print("Compressed Critical Point:   (%.4f, %.4f)" % (x_crit, y_crit))
print("Compressed Lambdas:          λ1=%.4f, λ2=%.4f, λ3=%.4f" % tuple(lambda_vec))
print("u: (%.4f, %.4f, %.4f)" % tuple(u_orig))
print("v: (%.4f, %.4f, %.4f)" % tuple(u_orig))
print("epsilon:", epsilon)
print("u':", u_opt)
print("v':", v_opt)
print("Optimization Success:", result.success)
print("Message:", result.message)
print("========== End of Summary ========")
# verify u' and v' to see if the critical point is outside the triangle
print("Verification:", verify(u_orig, v_orig, u_opt, v_opt, epsilon))



/project/xli281_uksr/mxia/apps/miniconda3/lib/python3.10/site-packages/scipy/optimize/_differentiable_functions.py:552: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  self.H.update(delta_x, delta_g)
/project/xli281_uksr/mxia/apps/miniconda3/lib/python3.10/site-packages/scipy/optimize/_differentiable_functions.py:317: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  self.H.update(self.x - self.x_prev, self.g - self.g_prev)


`xtol` termination condition is satisfied.
Number of iterations: 424, function evaluations: 5976, CG iterations: 1211, optimality: 4.79e-02, constraint violation: 1.67e-16, execution time:  1.1 s.

========== Summary ==========
Original Critical Point:     (0.1233, 0.5333)
Original Lambdas:            λ1=0.3433, λ2=0.1233, λ3=0.5333
Compressed Critical Point:   (0.3038, 1.0617)
Compressed Lambdas:          λ1=-0.3655, λ2=0.3038, λ3=1.0617
u: (1.1500, 0.3700, 0.2900)
v: (1.1500, 0.3700, 0.2900)
epsilon: 0.3
u': [1.14999999 0.36999999 0.29001917]
v': [-0.07       -1.25999999  0.33643748]
Optimization Success: True
Message: `xtol` termination condition is satisfied.
========== End of Summary ========
Critical point inside/outside mismatch.
Verification: False


In [67]:
#adaptive epsilon

import numpy as np
from scipy.optimize import minimize

def find_min_epsilon_feasible_solution(u_orig, v_orig, epsilon_start=0.3, epsilon_end=0.01, epsilon_step=0.01, verbose=False):
    """
    Adaptive epsilon search to find the smallest epsilon that allows
    a successful optimization (critical point pushed outside triangle and within error bounds).

    Parameters:
        u_orig (np.ndarray): Original u values at triangle vertices.
        v_orig (np.ndarray): Original v values at triangle vertices.
        epsilon_start (float): Starting epsilon value (max allowable perturbation).
        epsilon_end (float): Minimum epsilon to consider.
        epsilon_step (float): Step size to decrement epsilon.
        verbose (bool): Whether to print progress for each epsilon attempt.

    Returns:
        dict: Best feasible solution (smallest epsilon that succeeded),
              or last attempt if no feasible solution is found.
    """

    def barycentric_coords(x, y):
        return np.array([1 - x - y, x, y])

    def interpolate(vec, lambdas):
        return np.dot(lambdas, vec)

    def get_critical_point(u, v):
        A = np.array([[2, 1], [1, 2]])
        b = np.array([u[0] - u[1], v[0] - v[1]])
        x, y = np.linalg.solve(A, b)
        lambdas = barycentric_coords(x, y)
        return (x, y), lambdas

    def u_zero(z):
        u = z[0:3]
        x, y = z[6], z[7]
        lambdas = barycentric_coords(x, y)
        return interpolate(u, lambdas)

    def v_zero(z):
        v = z[3:6]
        x, y = z[6], z[7]
        lambdas = barycentric_coords(x, y)
        return interpolate(v, lambdas)

    def lambda_outside_constraint():
        return {'type': 'ineq', 'fun': lambda z: -np.min(barycentric_coords(z[6], z[7])) - 1e-5}

    def run_single_optimization(u_orig, v_orig, epsilon):
        (x0_cp, y0_cp), _ = get_critical_point(u_orig, v_orig)
        x0 = np.concatenate([u_orig, v_orig, [x0_cp, y0_cp]])

        bounds = [(u_orig[i] - epsilon, u_orig[i] + epsilon) for i in range(3)] + \
                 [(v_orig[i] - epsilon, v_orig[i] + epsilon) for i in range(3)] + \
                 [(-2.0, 2.0), (-2.0, 2.0)]

        constraints = [
            {'type': 'eq', 'fun': u_zero},
            {'type': 'eq', 'fun': v_zero},
            lambda_outside_constraint()
        ]

        result = minimize(
            fun=lambda z: np.sum(np.abs(z[0:3] - u_orig)) + np.sum(np.abs(z[3:6] - v_orig)),
            x0=x0,
            bounds=bounds,
            constraints=constraints,
            method='trust-constr',
            options={'disp': False, 'maxiter': 1000}
        )

        u_opt = result.x[0:3]
        v_opt = result.x[3:6]
        cp_opt = (result.x[6], result.x[7])
        lambda_opt = barycentric_coords(*cp_opt)
        cp_outside = np.min(lambda_opt) < -1e-3
        

        return {
            "epsilon": epsilon,
            "success": result.success,
            "cp_outside": cp_outside,
            "lambda": lambda_opt,
            "u_opt": u_opt,
            "v_opt": v_opt,
            "cp": cp_opt
        }

    history = []
    eps = epsilon_start
    while eps >= epsilon_end:
        result = run_single_optimization(u_orig, v_orig, eps)
        history.append(result)
        if verbose:
            print(f"ε = {eps:.3f} | success = {result['success']} | cp_outside = {result['cp_outside']} | λ_min = {np.min(result['lambda']):.4f}")
        if not result["success"] or not result["cp_outside"]:
            break
        eps -= epsilon_step

    return history[-2] if len(history) > 1 else history[-1]


# example usage
u_orig = [ 3.87, -9.42,  5.55]
v_orig = [ 5.23, 7.76,  2.53]
epsilon = 1.5

result = find_min_epsilon_feasible_solution(
    u_orig, v_orig,
    epsilon_start=epsilon,
    epsilon_end=0.01,
    epsilon_step=0.01,
    verbose=True
)

print("Best ε:", result["epsilon"])
print("Critical Point:", result["cp"])
print("λ:", result["lambda"])
print("Success:", result["success"])

/project/xli281_uksr/mxia/apps/miniconda3/lib/python3.10/site-packages/scipy/optimize/_differentiable_functions.py:552: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  self.H.update(delta_x, delta_g)
/project/xli281_uksr/mxia/apps/miniconda3/lib/python3.10/site-packages/scipy/optimize/_differentiable_functions.py:317: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  self.H.update(self.x - self.x_prev, self.g - self.g_prev)


ε = 1.500 | success = True | cp_outside = True | λ_min = -1.9944
ε = 1.490 | success = True | cp_outside = True | λ_min = -1.9971
ε = 1.480 | success = True | cp_outside = True | λ_min = -2.0000
ε = 1.470 | success = True | cp_outside = True | λ_min = -2.0000
ε = 1.460 | success = True | cp_outside = True | λ_min = -2.0000
ε = 1.450 | success = True | cp_outside = True | λ_min = -2.0000
ε = 1.440 | success = True | cp_outside = True | λ_min = -2.0000


/project/xli281_uksr/mxia/apps/miniconda3/lib/python3.10/site-packages/scipy/optimize/_trustregion_constr/equality_constrained_sqp.py:217: UserWarning: Singular Jacobian matrix. Using SVD decomposition to perform the factorizations.
  Z, LS, Y = projections(A, factorization_method)
/project/xli281_uksr/mxia/apps/miniconda3/lib/python3.10/site-packages/scipy/optimize/_trustregion_constr/equality_constrained_sqp.py:81: UserWarning: Singular Jacobian matrix. Using SVD decomposition to perform the factorizations.
  Z, LS, Y = projections(A, factorization_method)


ε = 1.430 | success = False | cp_outside = True | λ_min = -2.0041
Best ε: 1.44
Critical Point: (-0.6340195034674954, -1.9999962656915373)
λ: [ 3.63401577 -0.6340195  -1.99999627]
Success: True


In [ ]:
import numpy as np
from scipy.optimize import minimize

def find_min_epsilon_feasible_solution(u_orig, v_orig, epsilon_start=0.3, epsilon_end=0.01, epsilon_step=0.01, verbose=False):
    """
    Adaptive epsilon search to find the smallest epsilon that allows
    a successful optimization while preserving the original inside/outside status
    of the critical point with respect to a triangle.

    Parameters:
        u_orig (np.ndarray): Original u values at triangle vertices.
        v_orig (np.ndarray): Original v values at triangle vertices.
        epsilon_start (float): Starting epsilon value (max allowable perturbation).
        epsilon_end (float): Minimum epsilon to consider.
        epsilon_step (float): Step size to decrement epsilon.
        verbose (bool): Whether to print progress for each epsilon attempt.

    Returns:
        dict: Best feasible result (smallest epsilon that succeeds and matches original inside/outside status).
    """

    def barycentric_coords(x, y):
        return np.array([1 - x - y, x, y])

    def interpolate(vec, lambdas):
        return np.dot(lambdas, vec)

    def get_critical_point(u, v):
        A = np.array([[2, 1], [1, 2]])
        b = np.array([u[0] - u[1], v[0] - v[1]])
        x, y = np.linalg.solve(A, b)
        lambdas = barycentric_coords(x, y)
        return (x, y), lambdas

    def is_outside(lambda_vec, tol=1e-5):
        return (
            np.any(lambda_vec < -tol) or
            np.any(lambda_vec > 1 + tol) or
            not np.isclose(np.sum(lambda_vec), 1.0, atol=1e-5)
        )

    def u_zero(z):
        u = z[0:3]
        x, y = z[6], z[7]
        lambdas = barycentric_coords(x, y)
        return interpolate(u, lambdas)

    def v_zero(z):
        v = z[3:6]
        x, y = z[6], z[7]
        lambdas = barycentric_coords(x, y)
        return interpolate(v, lambdas)

    def lambda_outside_constraint():
        return {'type': 'ineq', 'fun': lambda z: -np.min(barycentric_coords(z[6], z[7])) - 1e-5}

    # Determine original CP position
    (_, _), lambda_orig = get_critical_point(u_orig, v_orig)
    orig_cp_outside = is_outside(lambda_orig)

    cp_orig, lambda_orig = get_critical_point(u_orig, v_orig)
    orig_cp_outside = is_outside(lambda_orig)

    if verbose:
        print("Original Critical Point: (%.4f, %.4f)" % cp_orig)
        print("Original Lambda: λ1=%.4f, λ2=%.4f, λ3=%.4f" % tuple(lambda_orig))
        print("Original CP Outside:", orig_cp_outside)

    def run_single_optimization(u_orig, v_orig, epsilon):
        (x0_cp, y0_cp), _ = get_critical_point(u_orig, v_orig)
        x0 = np.concatenate([u_orig, v_orig, [x0_cp, y0_cp]])

        bounds = [(u_orig[i] - epsilon, u_orig[i] + epsilon) for i in range(3)] + \
                 [(v_orig[i] - epsilon, v_orig[i] + epsilon) for i in range(3)] + \
                 [(-2.0, 2.0), (-2.0, 2.0)]

        constraints = [ # constraints for u and v to be zero at the critical point
            {'type': 'eq', 'fun': u_zero},
            {'type': 'eq', 'fun': v_zero}
        ]
        if orig_cp_outside:
            constraints.append(lambda_outside_constraint())

        result = minimize(
            fun=lambda z: np.sum(np.abs(z[0:3] - u_orig)) + np.sum(np.abs(z[3:6] - v_orig)),
            x0=x0,
            bounds=bounds,
            constraints=constraints,
            method='trust-constr', # 优化器：处理带有等式和不等式约束的问题
            options={'disp': False, 'maxiter': 1000}
        )

        u_opt = result.x[0:3]
        v_opt = result.x[3:6]
        cp_opt = (result.x[6], result.x[7])
        lambda_opt = barycentric_coords(*cp_opt)
        cp_outside = is_outside(lambda_opt)
        same_position = (cp_outside == orig_cp_outside)

        return {
            "epsilon": epsilon,
            "success": result.success,
            "cp_outside": cp_outside,
            "same_position": same_position,
            "lambda": lambda_opt,
            "u_opt": u_opt,
            "v_opt": v_opt,
            "cp": cp_opt
        }

    # Try first epsilon
    result_first = run_single_optimization(u_orig, v_orig, epsilon_start)
    if verbose:
        print(f"ε = {epsilon_start:.3f} | success = {result_first['success']} | cp_outside = {result_first['cp_outside']} | same_position = {result_first['same_position']} | λ_min = {np.min(result_first['lambda']):.4f}")

    if not result_first["success"] or not result_first["same_position"]:
        if verbose:
            print("Initial epsilon failed. Returning original u, v without modification.")
        return {
            "epsilon": 0.0,
            "success": True,
            "cp_outside": orig_cp_outside,
            "same_position": True,
            "lambda": lambda_orig,
            "u_opt": np.array(u_orig),
            "v_opt": np.array(v_orig),
            "cp": cp_orig
        }

    # Adaptive loop
    history = []
    eps = epsilon_start
    while eps >= epsilon_end:
        result = run_single_optimization(u_orig, v_orig, eps)
        history.append(result)
        if verbose:
            print(f"ε = {eps:.3f} | success = {result['success']} | cp_outside = {result['cp_outside']} | same_position = {result['same_position']} | λ_min = {np.min(result['lambda']):.4f} | λ_max = {np.max(result['lambda']):.4f}")
        if not result["success"] or not result["same_position"]:
            break
        eps -= epsilon_step

    return history[-2] if len(history) > 1 else history[-1]

# example usage outside
# u_orig = [ 3.87, -9.42,  5.55]
# v_orig = [ 5.23, 7.76,  2.53]
# epsilon = 1.5

#inside
u_orig = [1.65, 0.87, 0.79] #inside
v_orig = [-0.57, -1.76, 0.21] #inside
epsilon = 2

result = find_min_epsilon_feasible_solution(
    u_orig, v_orig,
    epsilon_start=epsilon,
    epsilon_end=0.01,
    epsilon_step=0.01,
    verbose=True
)

print("Best ε:", result["epsilon"])
print("Critical Point:", result["cp"])
print("λ:", result["lambda"])
print("Success:", result["success"])


Original Critical Point: (0.1233, 0.5333)
Original Lambda: λ1=0.3433, λ2=0.1233, λ3=0.5333
Original CP Outside: False
ε = 2.000 | success = True | cp_outside = True | same_position = False | λ_min = -0.9703
Initial epsilon failed. Returning original u, v without modification.
Best ε: 0.0
Critical Point: (0.1233333333333333, 0.5333333333333333)
λ: [0.34333333 0.12333333 0.53333333]
Success: True


In [30]:
# 先调u1v1，然后再调u2v2，然后再调u3v3
import numpy as np
from scipy.optimize import minimize

def optimize_vertex_sequentially(u_orig, v_orig, epsilon, verbose=True):
    def barycentric_coords(x, y):
        return np.array([1 - x - y, x, y])

    def compute_cp_and_lambda(u, v):
        A = np.array([[2, 1], [1, 2]])
        b = np.array([u[0] - u[1], v[0] - v[1]])
        x, y = np.linalg.solve(A, b)
        lambdas = barycentric_coords(x, y)
        return (x, y), lambdas

    def is_inside(lambda_vec, tol=1e-5):
        return (
            np.all(lambda_vec >= -tol)
            and np.all(lambda_vec <= 1 + tol)
            and np.isclose(np.sum(lambda_vec), 1.0, atol=1e-5)
        )

    # Copy original u and v to avoid modifying inputs
    u = u_orig.copy()
    v = v_orig.copy()

    # Compute original critical point and lambda
    cp0, lambda0 = compute_cp_and_lambda(u, v)
    if verbose:
        print(f"Original CP: {cp0}")
        print(f"Original lambda: {lambda0}")
        print(f"Inside: {is_inside(lambda0)}")

    # Sequential optimization over each vertex
    for idx in range(3):
        fixed = [i for i in range(3) if i != idx]
        lmb = lambda0.copy()

        # Optimize u[idx]
        def obj_u(x):
            temp_u = u.copy()
            temp_u[idx] = x[0]
            return (np.dot(lmb, temp_u))**2  # Make u(lambda) ≈ 0

        bounds_u = [(u_orig[idx] - epsilon, u_orig[idx] + epsilon)]
        res_u = minimize(obj_u, [u[idx]], bounds=bounds_u, method='L-BFGS-B')
        print("residual_u:", res_u.fun)
        u[idx] = res_u.x[0]

        # Optimize v[idx]
        def obj_v(x):
            temp_v = v.copy()
            temp_v[idx] = x[0]
            return (np.dot(lmb, temp_v))**2  # Make v(lambda) ≈ 0

        bounds_v = [(v_orig[idx] - epsilon, v_orig[idx] + epsilon)]
        res_v = minimize(obj_v, [v[idx]], bounds=bounds_v, method='L-BFGS-B') #如果无法在这个 ε 范围内找到使得插值为 0 的解，它就会卡在边界（达到 epsilon 限） this is memeory efficient
        print("residual_v:", res_v.fun)
        v[idx] = res_v.x[0]

        if verbose:
            print(f"\nAfter optimizing vertex {idx}:")
            print(f"u = {u}")
            print(f"v = {v}")
            cp_step, lambda_step = compute_cp_and_lambda(u, v)
            print(f"CP = {cp_step}")
            print(f"lambda = {lambda_step}")
            print(f"Inside: {is_inside(lambda_step)}")

    # Final result
    cp_final, lambda_final = compute_cp_and_lambda(u, v)
    return u, v, (cp_final, lambda_final), (is_inside(lambda0), is_inside(lambda_final))

u0 = np.array([1.65, 0.87, 0.79]) #inside
v0 = np.array([-0.57, -1.76, 0.21])

# u0 = np.array([ 3.87, -9.42,  5.55]) #outside
# v0 = np.array([ 5.23, 7.76,  2.53])


u_opt, v_opt, (cp_final, lambda_final),(lambda_ori,lambda_dec) = optimize_vertex_sequentially(u0, v0, epsilon=0.1)
if lambda_ori == lambda_dec:
    print("optimization success")
else:
    print("optimization failed")

# u_final_lambda = np.dot(lambda_final, u_opt)
# v_final_lambda = np.dot(lambda_final, v_opt)
# residual_norm = np.sqrt(u_final_lambda**2 + v_final_lambda**2)
# print(f"\nFinal residual norm at CP: {residual_norm:.4e}")



Original CP: (0.1233333333333333, 0.5333333333333333)
Original lambda: [0.34333333 0.12333333 0.53333333]
Inside: True
residual_u: 1.12529664
residual_v: 0.0709867211111111

After optimizing vertex 0:
u = [1.55 0.87 0.79]
v = [-0.47 -1.76  0.21]
CP = (0.023333333333333206, 0.6333333333333334)
lambda = [0.34333333 0.02333333 0.63333333]
Inside: True
residual_u: 1.0992823511111112
residual_v: 0.06456681

After optimizing vertex 1:
u = [1.55 0.77 0.79]
v = [-0.47 -1.66  0.21]
CP = (0.12333333333333324, 0.5333333333333333)
lambda = [0.34333333 0.12333333 0.53333333]
Inside: True
residual_u: 0.9902903511111113
residual_v: 0.040307254444444426

After optimizing vertex 2:
u = [1.55 0.77 0.69]
v = [-0.47 -1.66  0.31]
CP = (0.12333333333333324, 0.5333333333333333)
lambda = [0.34333333 0.12333333 0.53333333]
Inside: True
optimization success


In [ ]:

import numpy as np
from scipy.optimize import minimize
'''
一次性联合优化所有的 u 和 v
最小化lambda的残差=》 min(u(lambda’)^2, v(lambda’)^2)
明确检查原始 lambda 与当前 lambda 是否都 inside/outside 一致，并作为 constraint 加入
'''

# 这个只是用来修复一个三角形的，因为三角形之间有链接，所以可能需要多个patch同时修复，保证全局一致
# 但是如果全局丢进去优化的话变量太多会非常慢
# 也许可以考虑把数据集分成多个patch？（那patch之间咋处理？）
def is_inside(lambda_vec, tol=1e-5):
    return (
        np.all(lambda_vec >= -tol)
        and np.all(lambda_vec <= 1 + tol)
        and np.isclose(np.sum(lambda_vec), 1.0, atol=1e-5)
    )

def optimize_preserve_inside_status(u_orig, v_orig, epsilon=0.1, verbose=True):
    cp0, lambda0 = compute_cp_and_lambda(u_orig, v_orig)
    inside0 = is_inside(lambda0)
    
    def objective(x):
        u = x[:3]
        v = x[3:]
        _, lambda_ = compute_cp_and_lambda(u, v)
        u_lambda = np.dot(lambda_, u)
        v_lambda = np.dot(lambda_, v)
        return u_lambda**2 + v_lambda**2

    def inside_consistency_constraint(x):
        u = x[:3]
        v = x[3:]
        _, lambda_ = compute_cp_and_lambda(u, v)
        inside1 = is_inside(lambda_)
        # 如果原来在内，新的也必须在内 => return 1 if same else -1
        return 1.0 if inside1 == inside0 else -1.0

    bounds = [(u_orig[i] - epsilon, u_orig[i] + epsilon) for i in range(3)] + \
             [(v_orig[i] - epsilon, v_orig[i] + epsilon) for i in range(3)]

    x0 = np.concatenate([u_orig, v_orig])
    res = minimize(
        objective,
        x0,
        bounds=bounds,
        constraints=[{
            'type': 'ineq',
            'fun': inside_consistency_constraint
        }],
        method='SLSQP',
        options={'disp': verbose, 'ftol': 1e-12}
    )

    u_new, v_new = res.x[:3], res.x[3:]
    cp1, lambda1 = compute_cp_and_lambda(u_new, v_new)
    return u_new, v_new, lambda0, lambda1, is_inside(lambda0), is_inside(lambda1), res.success
# examples


print("========== test1 ==========")
u0 = np.array([1.65, 0.87, 0.79]) #inside
v0 = np.array([-0.57, -1.76, 0.21])
u_opt, v_opt, lambda0, lambda1, inside0, inside1, success = optimize_preserve_inside_status(u0, v0, epsilon=0.01)
print("u_orig:", u0)
print("u_opt:", u_opt)
print("v_orig:", v0)
print("v_opt:", v_opt)
print("diiference:", np.abs(u0 - u_opt), np.abs(v0 - v_opt))
print("lambda_orig:", lambda0)
print("lambda_opt:", lambda1)
print("inside_orig:", inside0)
print("inside_opt:", inside1)
print("Optimization success:", success)

print("========== test2 ==========")

u0 = np.array([ 3.87, -9.42,  5.55]) #outside
v0 = np.array([ 5.23, 7.76,  2.53])
u_opt, v_opt, lambda0, lambda1, inside0, inside1, success = optimize_preserve_inside_status(u0, v0, epsilon=0.01)
print("u_orig:", u0)
print("u_opt:", u_opt)
print("v_orig:", v0)
print("v_opt:", v_opt)
print("diiference:", np.abs(u0 - u_opt), np.abs(v0 - v_opt))
print("lambda_orig:", lambda0)
print("lambda_opt:", lambda1)
print("inside_orig:", inside0)
print("inside_opt:", inside1)
print("Optimization success:", success)

print("========== test3 ==========")
u0 = np.array([1.5, 0.7, 0.1])
v0= np.array([1.99,  -0.35, -5.0])
u_opt, v_opt, lambda0, lambda1, inside0, inside1, success = optimize_preserve_inside_status(u0, v0, epsilon=0.01)
print("u_orig:", u0)
print("u_opt:", u_opt)
print("v_orig:", v0)
print("v_opt:", v_opt)
print("diiference:", np.abs(u0 - u_opt), np.abs(v0 - v_opt))
print("lambda_orig:", lambda0)
print("lambda_opt:", lambda1)
print("inside_orig:", inside0)
print("inside_opt:", inside1)
print("Optimization success:", success)


========== test1 ==========
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.239450005555555
            Iterations: 2
            Function evaluations: 14
            Gradient evaluations: 2
u_orig: [1.65 0.87 0.79]
u_opt: [1.64 0.86 0.78]
v_orig: [-0.57 -1.76  0.21]
v_opt: [-0.56 -1.77  0.22]
diiference: [0.01 0.01 0.01] [0.01 0.01 0.01]
lambda_orig: [0.34333333 0.12333333 0.53333333]
lambda_opt: [0.33666667 0.11666667 0.54666667]
inside_orig: True
inside_opt: True
Optimization success: True


In [39]:
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import NonlinearConstraint
'''
一次性联合优化所有的 u 和 v
最小化lambda的残差=》 min(u(lambda’)^2, v(lambda’)^2)
明确检查原始 lambda 与当前 lambda 是否都 inside/outside 一致，并作为 constraint 加入
'''

# 这个只是用来修复一个三角形的，因为三角形之间有链接，所以可能需要多个patch同时修复，保证全局一致
# 但是如果全局丢进去优化的话变量太多会非常慢
# 也许可以考虑把数据集分成多个patch？（那patch之间咋处理？）
def is_inside(lambda_vec):
    return (
        np.all(lambda_vec >= 0)
        and np.all(lambda_vec <= 1)
        and np.isclose(np.sum(lambda_vec), 1.0)
    )
    
def barycentric_coords(x, y):
    return np.array([1 - x - y, x, y])

def compute_cp_and_lambda(u, v):
    A = np.array([[2, 1], [1, 2]])
    b = np.array([u[0] - u[1], v[0] - v[1]])
    x, y = np.linalg.solve(A, b)
    lambdas = barycentric_coords(x, y)
    return (x, y), lambdas

def optimize_move_cp_out(u_dec, v_dec, epsilon, verbose=True):
    cp0, lambda0 = compute_cp_and_lambda(u_dec, v_dec)
    inside0 = is_inside(lambda0)
    
    def objective(x):
        u = x[:3]
        v = x[3:]
        _, lambda_ = compute_cp_and_lambda(u, v)
        u_lambda = np.dot(lambda_, u)
        v_lambda = np.dot(lambda_, v)
        return u_lambda**2 + v_lambda**2

    def cp_move_outside_constraint(x):
        u = x[:3]
        v = x[3:]
        _, lambda_ = compute_cp_and_lambda(u, v)
        return 1.0 if not is_inside(lambda_) else -1.0
    def cp_move_outside_constraint_continuous(x):
        u = x[:3]
        v = x[3:]
        _, lambda_ = compute_cp_and_lambda(u, v)
        
        min_lambda = np.min(lambda_)
        max_lambda = np.max(lambda_)
        sum_lambda = np.sum(lambda_)

        # 只要 lambda 有一个 < 0，或者 > 1，就说明在外部
        # 也就是违反了 inside 的条件

        margin = max(-min_lambda, max_lambda - 1, np.abs(sum_lambda - 1))
        return margin
    

    bounds = [(u_dec[i] - epsilon, u_dec[i] + epsilon) for i in range(3)] + \
             [(v_dec[i] - epsilon, v_dec[i] + epsilon) for i in range(3)]

    x0 = np.concatenate([u_dec, v_dec])
    
    from scipy.optimize import NonlinearConstraint

    nonlinear_constraint = NonlinearConstraint(
        cp_move_outside_constraint_continuous,
        lb=1e-3,  # constraint must be ≥ 1e-6
        ub=np.inf
    )

    res = minimize(
        objective,
        x0,
        method='trust-constr',
        bounds=bounds,
        constraints=[nonlinear_constraint],
        options={'verbose': 1}
    )

    # res = minimize(
    #     objective,
    #     x0,
    #     bounds=bounds,
    #     constraints=[{
    #         'type': 'ineq',
    #         # 'fun': cp_move_outside_constraint
    #         'fun': lambda x: cp_move_outside_constraint_continuous(x) - 1e-6
    #     }],
    #     # method='SLSQP',
    #     method='trust-constr',
    #     options={'disp': verbose}
    # )

    u_new, v_new = res.x[:3], res.x[3:]
    cp1, lambda1 = compute_cp_and_lambda(u_new, v_new)
    return u_new, v_new, lambda0, lambda1, is_inside(lambda0), is_inside(lambda1), res.success
# examples


print("========== test1 ==========")
u0 = np.array([1.65, 0.87, 0.79]) #inside
v0 = np.array([-0.57, -1.76, 0.21])
u_opt, v_opt, lambda0, lambda1, inside0, inside1, success = optimize_move_cp_out(u0, v0, epsilon=0.05)
print("u_dec:", u0)
print("u_opt:", u_opt)
print("v_dec:", v0)
print("v_opt:", v_opt)
print("diiference:", np.abs(u0 - u_opt), np.abs(v0 - v_opt))
print("lambda_orig:", lambda0)
print("lambda_opt:", lambda1)
print("inside_orig:", inside0)
print("inside_opt:", inside1)
print("Optimization success:", success)

# print("========== test2 ==========")

# u0 = np.array([ 3.87, -9.42,  5.55]) #outside
# v0 = np.array([ 5.23, 7.76,  2.53])
# u_opt, v_opt, lambda0, lambda1, inside0, inside1, success = optimize_preserve_inside_status(u0, v0, epsilon=0.01)
# print("u_dec:", u0)
# print("u_opt:", u_opt)
# print("v_dec:", v0)
# print("v_opt:", v_opt)
# print("diiference:", np.abs(u0 - u_opt), np.abs(v0 - v_opt))
# print("lambda_orig:", lambda0)
# print("lambda_opt:", lambda1)
# print("inside_orig:", inside0)
# print("inside_opt:", inside1)
# print("Optimization success:", success)

# print("========== test3 ==========")
# u0 = np.array([1.5, 0.7, 0.1])
# v0= np.array([1.99,  -0.35, -5.0])
# u_opt, v_opt, lambda0, lambda1, inside0, inside1, success = optimize_preserve_inside_status(u0, v0, epsilon=0.01)
# print("u_dec:", u0)
# print("u_opt:", u_opt)
# print("v_dec:", v0)
# print("v_opt:", v_opt)
# print("diiference:", np.abs(u0 - u_opt), np.abs(v0 - v_opt))
# print("lambda_orig:", lambda0)
# print("lambda_opt:", lambda1)
# print("inside_orig:", inside0)
# print("inside_opt:", inside1)
# print("Optimization success:", success)


========== test1 ==========
Constraint violation exceeds 'gtol'
Number of iterations: 389, function evaluations: 4494, CG iterations: 413, optimality: 6.44e-15, constraint violation: 1.00e-03, execution time: 0.73 s.
u_dec: [1.65 0.87 0.79]
u_opt: [1.6        0.82000001 0.74      ]
v_dec: [-0.57 -1.76  0.21]
v_opt: [-0.52       -1.81        0.25999999]
diiference: [0.05       0.04999999 0.05      ] [0.05       0.05       0.04999999]
lambda_orig: [0.34333333 0.12333333 0.53333333]
lambda_opt: [0.31 0.09 0.6 ]
inside_orig: True
inside_opt: True
Optimization success: False


In [52]:
import cvxpy as cp
import numpy as np

def is_inside(lambda_vec, tol=1e-6):
    return (
        np.all(lambda_vec >= -tol) and
        np.all(lambda_vec <= 1 + tol) and
        np.isclose(np.sum(lambda_vec), 1.0, atol=tol)
    )

def barycentric_coords(x, y):
    return np.array([1 - x - y, x, y])

def compute_lambda(u, v):
    A = np.array([[2, 1], [1, 2]])
    b = np.array([u[0] - u[1], v[0] - v[1]])
    x, y = np.linalg.solve(A, b)
    return barycentric_coords(x, y)

def solve_lp_with_milp_lambda_outside(u_orig, v_orig, u_dec, v_dec, epsilon):
    u = cp.Variable(3)
    v = cp.Variable(3)

    # Objective: stay close to decompressed data
    objective = cp.Minimize(cp.norm1(u - u_dec) + cp.norm1(v - v_dec))

    constraints = [
        u >= u_orig - epsilon,
        u <= u_orig + epsilon,
        v >= v_orig - epsilon,
        v <= v_orig + epsilon
    ]

    # Manually solve for lambda
    du = u[0] - u[1]
    dv = v[0] - v[1]
    x = (2 * du - dv) / 3
    y = (-du + 2 * dv) / 3
    lambda_vec = cp.hstack([1 - x - y, x, y])

    # Binary variables to enforce at least one lambda out of [0,1]
    z = cp.Variable(3, boolean=True)
    M = 10.0  # Big-M
    z_pairs = []
    tau = 1e-3  # Small tolerance for numerical stability

    for i in range(3):
        z_low = cp.Variable(boolean=True)
        z_high = cp.Variable(boolean=True)
        z_pairs.append((z_low, z_high))

        constraints += [
            lambda_vec[i] <= -tau + M * (1 - z_low),
            lambda_vec[i] >= 1 + tau - M * (1 - z_high),
            z_low + z_high <= 1  # 只能向下或向上离开，不同时
        ]
    
    # Enforce at least one z_i == 1 → at least one lambda out-of-bounds
    constraints.append(cp.sum([z_low + z_high for (z_low, z_high) in z_pairs]) >= 1)

    # Solve MILP
    prob = cp.Problem(objective, constraints)
    prob.solve(solver=cp.SCIPY, verbose=True)  # You can also try 'ECOS_BB' or 'SCIP'

    print(f"Status: {prob.status}")
    print("u_opt:", u.value)
    print("v_opt:", v.value)
    return u.value, v.value

# Example test
u_orig = np.array([1.65, 0.87, 0.79])
v_orig = np.array([-0.57, -1.76, 0.21])
u_dec = u_orig + np.random.uniform(-0.03, 0.03, size=3)
v_dec = v_orig + np.random.uniform(-0.03, 0.03, size=3)
epsilon = 0.05

u_opt, v_opt = solve_lp_with_milp_lambda_outside(u_orig, v_orig, u_dec, v_dec, epsilon)
print("opt barycentric_coords:", compute_lambda(u_opt, v_opt))
print("original barycentric_coords:", compute_lambda(u_orig, v_orig))
print("decompressed barycentric_coords:", compute_lambda(u_dec, v_dec))


                                     CVXPY                                     
                                     v1.6.5                                    
(CVXPY) Apr 21 01:38:31 PM: Your problem has 12 variables, 22 constraints, and 0 parameters.
(CVXPY) Apr 21 01:38:31 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Apr 21 01:38:31 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Apr 21 01:38:31 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Apr 21 01:38:31 PM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Apr 21 01:38:31 PM: Compiling problem (target solver=SCIPY).
(C

(CVXPY) Apr 21 01:38:31 PM: Applying reduction CvxAttr2Constr
(CVXPY) Apr 21 01:38:31 PM: Applying reduction ConeMatrixStuffing
(CVXPY) Apr 21 01:38:31 PM: Applying reduction SCIPY
(CVXPY) Apr 21 01:38:31 PM: Finished problem compilation (took 2.882e-02 seconds).
-------------------------------------------------------------------------------
                                Numerical solver                               
-------------------------------------------------------------------------------
(CVXPY) Apr 21 01:38:31 PM: Invoking solver SCIPY  to obtain a solution.
Solver terminated with message: The problem is infeasible. (HiGHS Status 8: model_status is Infeasible; primal_status is None)
-------------------------------------------------------------------------------
                                    Summary                                    
-------------------------------------------------------------------------------
(CVXPY) Apr 21 01:38:31 PM: Problem status: infeasible
(

TypeError: 'NoneType' object is not subscriptable

In [24]:
import cvxpy as cp
import numpy as np

def compute_lambda(u, v):
    A = np.array([[2, 1], [1, 2]])
    b = np.array([u[0] - u[1], v[0] - v[1]])
    x, y = np.linalg.solve(A, b)
    return np.array([1 - x - y, x, y])

def solve_lp_with_milp_lambda_outside(u_orig, v_orig, u_dec, v_dec, epsilon=0.05, tau=1e-4, M=10.0):
    u = cp.Variable(3)
    v = cp.Variable(3)

    # Objective: stay close to decompressed data
    objective = cp.Minimize(cp.norm1(u - u_dec) + cp.norm1(v - v_dec))

    # constraints = [
    #     u >= u_orig - epsilon, # 这里不该用orig啊，因为不知道原始数据
    #     u <= u_orig + epsilon,
    #     v >= v_orig - epsilon,
    #     v <= v_orig + epsilon
    # ]
    constraints = [
        u >= u_dec - epsilon, # 这里不该用orig啊，因为不知道原始数据
        u <= u_dec + epsilon,
        v >= v_dec - epsilon,
        v <= v_dec + epsilon
    ]

    # Compute lambda: critical point location in barycentric coords
    du = u[0] - u[1]
    dv = v[0] - v[1]
    x = (2 * du - dv) / 3
    y = (-du + 2 * dv) / 3
    lambda_vec = cp.hstack([1 - x - y, x, y])

    # Binary variables for MILP: enforce lambda out-of-bounds
    z_low = [cp.Variable(boolean=True) for _ in range(3)]
    z_high = [cp.Variable(boolean=True) for _ in range(3)]

    for i in range(3):
        constraints += [
            lambda_vec[i] <= -tau + M * (1 - z_low[i]),    # lambda_i < 0 if z_low[i] == 0
            lambda_vec[i] >= 1 + tau - M * (1 - z_high[i]),# lambda_i > 1 if z_high[i] == 0
            z_low[i] + z_high[i] <= 1                      # can't be both
        ]

    # Require at least one of the three lambdas to be out of bounds
    constraints.append(cp.sum(z_low + z_high) >= 1)

    prob = cp.Problem(objective, constraints)
    #prob.solve(verbose=True,solver = cp.GUROBI)
    prob.solve(verbose=True)

    u_opt = u.value
    v_opt = v.value
    lambda_opt = compute_lambda(u_opt, v_opt)

    return u_opt, v_opt, lambda_opt

# Example test
if __name__ == "__main__":
    # np.random.seed(0)
    '''
    u_orig = np.array([1.65, 0.87, 0.79])
    v_orig = np.array([-0.57, -1.76, 0.21])
    # u_dec= [1.6988135 , 1.08518937 ,0.89276338]  #max 0.5
    # v_dec= [-0.52511682 ,-1.8363452 ,  0.35589411]
    u_dec=[1.57742758, 0.93696415, 0.64271914]  #max 0.2,diff=[0.148,0.127]
    v_dec=[-0.48346912, -1.84423756,  0.08327654]
    print("max perturbation:", np.max(np.abs(u_orig - u_dec)), np.max(np.abs(v_orig - v_dec)))
    epsilon = 0.2 #eps between ori and opt
    '''
    u_orig = np.array([-0.3936493702649537, -1.1167765341060027, 0.6712816344005144]) #outside
    v_orig = np.array([0.21612322268661188, -1.3122574878632185, 1.3086441619212135]) #outside
    u_dec = np.array([-0.3668775379175536,-1.1315609638259452,0.6981201536030845])  #inside dec data ,errormax=0.03
    v_dec = np.array([0.205286096405359,-1.3155794907751646,1.3223926165898532]) #inside
    print("ori_dec_errors:", np.abs(u_orig - u_dec), np.abs(v_orig - v_dec))
    epsilon = 0.03 #eps between ori and opt

    u_opt, v_opt, lambda_opt = solve_lp_with_milp_lambda_outside(u_orig, v_orig, u_dec, v_dec, epsilon)
    print("u_orig:", u_orig, "v_orig:", v_orig)
    print("original lambda:", compute_lambda(u_orig, v_orig))
    print("u_dec:", u_dec, "v_dec:", v_dec)
    print("decompressed lambda:", compute_lambda(u_dec, v_dec))
    print("opt_ori_errors:", np.abs(u_orig - u_opt), np.abs(v_orig - v_opt))
    print("=== Result ===")
    print("u_opt:", u_opt)
    print("v_opt:", v_opt)
    print("lambda_opt:", lambda_opt)
    print("inside_opt:", np.all(lambda_opt >= 0) and np.all(lambda_opt <= 1) and np.isclose(np.sum(lambda_opt), 1.0))



ori_dec_errors: [0.02677183 0.01478443 0.02683852] [0.01083713 0.003322   0.01374845]
                                     CVXPY                                     
                                     v1.6.5                                    
(CVXPY) Apr 21 05:34:40 PM: Your problem has 12 variables, 22 constraints, and 0 parameters.
(CVXPY) Apr 21 05:34:40 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Apr 21 05:34:40 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Apr 21 05:34:40 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Apr 21 05:34:40 PM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                                  
-----------------------------------------------------------------

In [1]:
import pyvista as pv
import numpy as np
height, width = 2400, 3600
# 读取 .vtp 文件
mesh_ori = pv.read("uv_mesh.vtp")
mesh_ori_u = np.fromfile("uf.dat", dtype=np.float32).reshape((height, width))
mesh_ori_v = np.fromfile("vf.dat", dtype=np.float32).reshape((height, width))

# # 查看信息
# print(mesh)
# print("Number of points:", mesh.n_points)
# print("Number of cells:", mesh.n_cells)
# print("Available point data arrays:", mesh.point_data.keys())
# print("Available cell data arrays:", mesh.cell_data.keys())
mesh_sz3 = pv.read("uv_mesh_sz3.vtp")
mesh_sz3_u = np.fromfile("uf.sz3.out", dtype=np.float32).reshape((height, width))
mesh_sz3_v = np.fromfile("vf.sz3.out", dtype=np.float32).reshape((height, width))

# check max diff
diff_u = np.abs(mesh_ori_u - mesh_sz3_u)
diff_v = np.abs(mesh_ori_v - mesh_sz3_v)
print("max diff u:", np.max(diff_u))
print("max diff v:", np.max(diff_v))

# # 查看信息
#print max values
# print("max u_ori:", np.max(mesh_ori_u))
# print("max v_ori:", np.max(mesh_ori_v))
# print("max u_sz3:", np.max(mesh_sz3_u))
# print("max v_sz3:", np.max(mesh_sz3_v))

# print min values
# print("min u_ori:", np.min(mesh_ori_u))
# print("min v_ori:", np.min(mesh_ori_v))
# print("min u_sz3:", np.min(mesh_sz3_u))
# print("min v_sz3:", np.min(mesh_sz3_v))


max diff u: 0.099999964
max diff v: 0.099999905
